### CARREGA BIBLIOTECAS

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier

#%load_ext nb_black


### CARREGA DATASET

In [2]:
heart_data = pd.read_csv('heart_failure_clinical_records_dataset.csv')
heart_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [4]:
heart_data.nunique().sort_values()

anaemia                       2
diabetes                      2
high_blood_pressure           2
sex                           2
smoking                       2
DEATH_EVENT                   2
ejection_fraction            17
serum_sodium                 27
serum_creatinine             40
age                          47
time                        148
platelets                   176
creatinine_phosphokinase    208
dtype: int64

Se você tivesse muitas colunas e tivesse que definir arbitrariamente um número máximo 
de valores únicos que determinaria se aquela feature é ou não numérica, você poderia 
selecionar as features da seguinte forma:

```python
features_categoricas = df.loc[:, df.nunique() < N].columns
```

O código acima determina que todas as colunas com menos de N valores únicos são
categóricas. De forma análoga, você poderia inverter o símbolo e filtrar as colunas numéricas.

No nosso caso, as features categóricas são dummies / binárias (0 ou 1). Sendo assim, 
podemos tratá-las como numéricas também. Ou seja, nosso dataset  só possui features numéricas.

Porém, para deixar o exercício mais interessante, vamos deixar a feature <i>diabetes</i> como categórica.


In [5]:
#heart_data['diabetes'] = heart_data['diabetes'].map({1: 'yes', 0: 'no'})

In [6]:
heart_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [7]:
# Checando a mudança:
heart_data['diabetes'].value_counts()

0    174
1    125
Name: diabetes, dtype: int64

In [8]:
heart_data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


Agora, nosso dataset possui uma feature categórica e o resto numérica. Sem mais delongas, 
vamos à modelagem!

In [9]:
heart_data.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time',
       'DEATH_EVENT'],
      dtype='object')

### MODELO DE MACHINE LEARNING

In [10]:
features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
            'ejection_fraction', 'high_blood_pressure', 'platelets',
            'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']

target = 'DEATH_EVENT'

In [11]:
X = heart_data[features]
y = heart_data[target]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [13]:
categorical_features = ['diabetes']
numerical_features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
                      'ejection_fraction', 'high_blood_pressure', 'platelets',
                      'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']



Aproveitando, imagina que você tivesse outras duas variáveis:
- Variável 1: Classe Social, valores que ela recebe: 1,2,3,4,5.
- Variável 2: Estado, recebe SP, RJ, DF.
- Variável 3: Latitude, recebe valores da latitude.

Duas coisas, vale pontuar:
- A variável 1 ainda precisa ser tratada como categórica. Ela veio no formato numérico, mas ela é uma categoria.
  Se você tratar como numérico, você estaria se equivocando. 
  A classe 1 sendo a classe A e a classe 5 sendo a classe E não possuem 4 de distância, são apenas categorias diferentes.
- Inclusive, aproveitando o gancho, aí está o perigo de, por exemplo, você usar o Ordinal Encoder para tratar a variável 2.
  Aqui, veja que sequer existe uma ordenação. Então chamar SP de 1, RJ de 2 e DF de 3 não faz muito sentido. Por que SP e DF teriam 2 de distância e SP e RJ teriam 1? Por que a distância de SP para DF seriam o dobro de unidades de SP e RJ? Entende meu ponto?
- Novamente, aqui temos uma confusão de olhar para uma categoria e tratar como numérica. Você não pode apenas usar uma latitude como numérica, porque ela na verdade é um local.

COMO APLICAR TRATAMENTOS ESPECÍFICOS PARA CADA COLUNA?

```python
transformer = ColumnTransformer([
    ('nome_da_transformacao', transformacao_a_ser_feita, colunas_afetadas),
    ('nome_da_transformacao1', transformacao_a_ser_feita1, colunas_afetadas1),
    ('nome_da_transformacao2', transformacao_a_ser_feita2, colunas_afetadas2),
    .
    .
    .
    ('nome_da_transformacaoN', transformacao_a_ser_feitaN, colunas_afetadasN)
])

transformer.fit_transform(X_train)
transformer.transform(X_test)
```


In [14]:
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', ce.TargetEncoder()),
    
])
transformer = ColumnTransformer([
  ('categorical_transformer', categorical_pipe, categorical_features),
  ('numerical_transformer', SimpleImputer(strategy='median'), numerical_features)
 ])

X_train_transformed = transformer.fit_transform(X_train, y_train)
X_test_transformed= transformer.transform(X_test)

In [15]:
heart_data.head(10)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
5,90.0,1,47,0,40,1,204000.00,2.1,132,1,1,8,1
6,75.0,1,246,0,15,0,127000.00,1.2,137,1,0,10,1
7,60.0,1,315,1,60,0,454000.00,1.1,131,1,1,10,1
8,65.0,0,157,0,65,0,263358.03,1.5,138,0,0,10,1
9,80.0,1,123,0,35,1,388000.00,9.4,133,1,1,10,1


In [17]:
tree = DecisionTreeClassifier()
tree.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [20]:
y_pred = tree.predict(X_test_transformed)

In [21]:
y_pred

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [22]:
y_test.values

array([0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1])

Chegou um dataset novo, i.e, novos clientes, o que fazer ?

Você precisa tratar e aplicar o modelo neles, assim como fizemos com os dados de teste.

De forma resumida, seria algo assim:

```python
# Voce carregaria os dados
    novos_pacientes = pd.read_csv('novos_pacientes,csv')

# Teria que filtrar as colunas que usamos no modelo
    novos_pacientes_2 = novos_pacientes[features]

# Aplicar as trasnformacoes
    transformer.transform(novos_pacientes_2)

# Fazer a predicao final
    predicao_final = tree.predict(novos_pacientes_2)

# A predicao final contem as informacoes sobre esses novos pacientes terem maior ou menor propensao a morrer.
```



In [24]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

print(f" Acurácia: {accuracy_score(y_test, y_pred):.2f}")
print(f" Recall: {recall_score(y_test, y_pred):.2f}")
print(f" Precision: {precision_score(y_test, y_pred):.2f}")
print(f" F1-Score: {f1_score(y_test, y_pred):.2f}")
print(f" ROC/AUC: {roc_auc_score(y_test, y_pred):.2f}")

 Acurácia: 0.65
 Recall: 0.44
 Precision: 0.61
 F1-Score: 0.51
 ROC/AUC: 0.62


Precision = TP/(TP + FP)

Recall = TP/(TP + FN)